Importing necessary libraries

In [35]:
import random
from SPARQLWrapper import SPARQLWrapper, JSON
from datasets import Dataset, DatasetDict

Make code reproducible

In [36]:
random.seed(50)

This function helps us to transform result of a query into a list so it is easier to use.

In [37]:
def get_entities(sparql_query):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return [r[next(iter(r))]["value"] for r in results["results"]["bindings"]]

Getting 20000 mountain names from Wikipedia to work with

In [38]:
query = """
SELECT ?mountainLabel WHERE {
  ?mountain wdt:P31 wd:Q8502.  # instance of: mountain
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 20000
"""
mountains = get_entities(query)

Creating sentences with mountain name. 4 sentences for each mountain name, so we have 80000 sentences with one mountain name

In [39]:
# 125 templates (most generated by ChatGPT) for creating sentences with one mountain name
mountains_template = [
    "{m} is the highest peak in its region.",
    "Mountaineers often visit {m} during the summer.",
    "It is so amazing to look down from {m}.",
    "{m} can serve as a natural barrier during war.",
    "The climb up {m} was challenging due to heavy snow.",
    "Beautiful animals live on {m}.",
    "You need to visit {m} at least one time in your life!",
    "You can see {m} from far away.",
    "There is village near {m}.",
    "Many people tried to climb the {m}, but not everyone succeeded.",
    "It is very cold on top of {m}.",
    "My parents met on {m} during a hike.",
    "{m} attracts thousands of tourists every year.",
    "The air around {m} is incredibly fresh and clean.",
    "Legends say that gods once lived on {m}.",
    "From the top of {m}, you can see the sunrise like nowhere else.",
    "Climbers must prepare carefully before ascending {m}.",
    "Snow covers {m} for most of the year.",
    "The landscape around {m} changes dramatically with the seasons.",
    "Scientists often study glaciers located on {m}.",
    "Few people have reached the summit of {m}.",
    "{m} has inspired artists and poets for centuries.",
    "Storms near {m} can appear suddenly and violently.",
    "The route to {m} passes through dense forests.",
    "Local communities depend on the rivers that flow from {m}.",
    "{m} is located in one of the most scenic regions of the world.",
    "The slopes of {m} are covered with dense forests.",
    "Rivers originating from {m} provide water to nearby villages.",
    "The valleys surrounding {m} are popular hiking destinations.",
    "{m} is surrounded by smaller hills and ridges.",
    "From {m}, you can see the neighboring mountain ranges.",
    "The terrain around {m} changes dramatically from base to peak.",
    "Rock formations on {m} attract geologists from around the world.",
    "The cliffs of {m} are known for their unusual shapes.",
    "Many natural caves can be found on {m}.",
    "{m} is part of a protected nature reserve.",
    "Several lakes are located at the foot of {m}.",
    "The summit of {m} is often covered in clouds.",
    "{m} features steep slopes and jagged peaks.",
    "Sandstone formations on {m} create a unique landscape.",
    "Adventure seekers often come to {m} for climbing.",
    "Hiking trails on {m} are well-marked and maintained.",
    "Camping near {m} is popular during summer months.",
    "Skiers enjoy the slopes of {m} in winter.",
    "Tourists often take photographs at sunrise on {m}.",
    "Paragliders use {m} as a launch point.",
    "There are guided tours available for visitors to {m}.",
    "Mountain lodges near {m} offer a cozy stay.",
    "Backpackers explore the hidden paths of {m}.",
    "Cyclists often ride along the lower trails of {m}.",
    "Rock climbing courses are organized on {m}.",
    "Birdwatchers visit {m} to observe rare species.",
    "Many tourists trek to the base of {m} every year.",
    "Photographers love capturing the mist around {m}.",
    "Adventure camps are sometimes held on {m}.",
    "The weather on {m} can change rapidly.",
    "Snowstorms on {m} are common during winter.",
    "High winds sweep across the peak of {m}.",
    "Fog often covers the lower slopes of {m}.",
    "Lightning sometimes strikes the cliffs of {m}.",
    "The air on {m} is cold and crisp.",
    "Rainfall on {m} nourishes nearby forests.",
    "The temperature drops significantly at the top of {m}.",
    "Heavy snow can block paths on {m}.",
    "Sunshine makes the summit of {m} sparkle in the morning.",
    "Climbers prepare carefully before attempting {m}.",
    "Many mountaineers have set records on {m}.",
    "Rescue operations are sometimes required on {m}.",
    "The ascent of {m} can be dangerous for inexperienced hikers.",
    "Experienced guides help tourists climb {m} safely.",
    "Several routes lead to the top of {m}.",
    "Mountaineering schools use {m} for training purposes.",
    "Climbers must carry sufficient gear when ascending {m}.",
    "Some sections of {m} require ropes and harnesses.",
    "Many adventurers document their climbs on {m} online.",
    "Rare birds nest in the forests of {m}.",
    "Mountain goats are often seen on the cliffs of {m}.",
    "Wildflowers bloom abundantly around {m} in spring.",
    "Bears occasionally roam the lower regions of {m}.",
    "The ecosystem of {m} is diverse and fragile.",
    "Insects and small mammals thrive on {m}.",
    "Streams on {m} support fish and amphibians.",
    "The flora on {m} changes with altitude.",
    "Butterflies are commonly spotted around {m} during summer.",
    "Conservationists study animals living near {m}.",
    "Ancient travelers wrote about {m} in their journals.",
    "Legends say that gods once lived on {m}.",
    "Local communities have cultural festivals near {m}.",
    "Historical battles were sometimes fought near {m}.",
    "Old temples can be found on the slopes of {m}.",
    "People have lived near {m} for centuries.",
    "Folklore often mentions {m} as a sacred place.",
    "Explorers documented their journeys to {m} in maps.",
    "Traditions in nearby villages are influenced by {m}.",
    "Some mountains, like {m}, are important pilgrimage sites.",
    "Geologists study the rock formations of {m}.",
    "Scientists measure the elevation of {m} regularly.",
    "Glaciers on {m} provide climate data.",
    "Researchers study the biodiversity around {m}.",
    "Seismic activity near {m} is monitored carefully.",
    "Environmentalists monitor pollution around {m}.",
    "Meteorologists study weather patterns on {m}.",
    "The soil composition on {m} is unique and varied.",
    "Scientists map the terrain of {m} using satellites.",
    "Ecologists conduct field studies on {m} each season.",
    "{m} is famous for its breathtaking cliffs.",
    "Many hikers take photos at the base of {m}.", 
    "{m} has peaks that are covered with snow year-round.",  
    "Local legends tell stories about {m}.",
    "The view from the top of {m} is unforgettable.",
    "{m} is home to many rare plants.",
    "Adventurers often plan expeditions to {m}.",
    "Several rivers originate from {m}.",  
    "Camping near {m} is a favorite activity for tourists.",  
    "{m} has been explored by mountaineers from all over the world.",  
    "Birds often nest in the forests on {m}.",  
    "The weather on {m} can change very quickly.",  
    "Scientists study the glaciers and rocks on {m}.",  
    "Sunrise seen from {m} is a spectacular sight.",  
    "{m} is considered sacred by the local population.",  
    "Climbers need special gear to ascend {m} safely.",  
    "Trails leading to {m} are challenging but rewarding.",  
    "Wild animals roam freely on {m}.",  
    "The terrain of {m} varies from rocky slopes to grassy meadows.",  
    "Tourists from many countries visit {m} every year."
]

In [40]:
dataset = []

# For each mountain 4 random templates are choosed. So we have 80000 sentences in total
for m in mountains:
    for t in random.sample(mountains_template, 4):
        text = t.format(m=m)
        start = text.index(m)
        end = start + len(m)
        dataset.append({
            "text": text,
            "entities": [{"start": start, "end": end, "label": "MOUNTAIN"}]
        })

We also must create sentences with multiple mountain names occur

In [41]:
# 60 templates (most generated by ChatGPT) for creating sentences with two mountain names
two_mountains_template = [
    "Are {m1} higher than {m2}?",
    "I climbed the {m1} last month, so {m2} is my next goal.",
    "In my opinion, {m1} is more beautiful than {m2}.",
    "I recommend you to visit {m1} and {m2} this year.",
    "I couldn't climb up {m1} earlier, but now I am on {m2}.",
    "Every rock climber should now about {m1} and {m2}.",
    "Both {m1} and {m2} attract many climbers from around the world.",
    "The route between {m1} and {m2} offers breathtaking views.",
    "Many expeditions start at {m1} and finish near {m2}.",
    "While {m1} is covered with snow, {m2} often stays dry and rocky.",
    "Legends say a hidden path connects {m1} and {m2}.",
    "Weather conditions on {m1} and {m2} can change in just a few hours.",
    "Photographers love capturing sunsets over {m1} and {m2}.",
    "Rivers flowing from {m1} and {m2} join together in the same valley.",
    "The view from {m1} is different from that of {m2}.",
    "Climbers often compare the difficulty of {m1} and {m2}.",
    "The peaks of {m1} and {m2} are visible from the valley below.",
    "Animals migrate between {m1} and {m2} during the winter.",
    "Hiking from {m1} to {m2} takes several days.",
    "Both {m1} and {m2} have unique rock formations.",
    "Many photographers plan trips to capture both {m1} and {m2}.",
    "The rivers flowing from {m1} and {m2} are vital to local agriculture.", 
    "Historical records mention expeditions to {m1} and {m2}.",  
    "{m1} is taller than {m2}, but {m2} is steeper.",  
    "Tourists love visiting {m1} in the morning and {m2} in the evening.",  
    "Snow covers {m1} longer than {m2} each year.",  
    "There are hiking trails connecting {m1} and {m2}.",  
    "Geologists study the formations of {m1} and {m2} together.",  
    "The air is crisp at both {m1} and {m2}.",  
    "Local legends link {m1} and {m2} as sacred mountains.",  
    "Weather patterns are different on {m1} compared to {m2}.",  
    "Some climbers attempt to summit {m1} and {m2} in a single trip.",  
    "Wildflowers bloom on {m1} earlier than on {m2}.",  
    "Guides often recommend visiting {m1} before {m2}.",  
    "The sunlight hits {m1} and {m2} in contrasting ways at sunrise.",  
    "Trekkers document routes involving both {m1} and {m2}.",  
    "The paths around {m1} are more challenging than those near {m2}.",  
    "Many animals find shelter between {m1} and {m2}.",  
    "The local community celebrates festivals at both {m1} and {m2}.",  
    "Camping sites are available on {m1} and {m2} for hikers.",  
    "Rivers that start from {m1} and {m2} meet downstream.",  
    "Storms on {m1} can affect climbers on {m2}.",  
    "Trekkers often photograph both {m1} and {m2} from a single viewpoint.",  
    "Maps of the region show both {m1} and {m2} prominently.",  
    "The slopes of {m1} and {m2} are home to diverse wildlife.",  
    "Legends claim a secret path runs between {m1} and {m2}.",  
    "Scientists measure the height and slope of both {m1} and {m2}.",  
    "Tourists often plan to visit {m1} and {m2} in the same trip.",  
    "While {m1} has glaciers, {m2} has rocky peaks.",  
    "The forests surrounding {m1} and {m2} are protected areas.",  
    "Climbing equipment is required for both {m1} and {m2}.",  
    "Several expeditions were carried out simultaneously on {m1} and {m2}.",  
    "The base of {m1} is easier to reach than the base of {m2}.",  
    "Snowmelt from {m1} and {m2} nourishes nearby fields.",  
    "Adventurers often plan to summit {m1} first and {m2} second.",  
    "The peaks of {m1} and {m2} look spectacular from the nearby lake.",  
    "Local guides know hidden trails on {m1} and {m2}.",  
    "The terrain between {m1} and {m2} is rugged and challenging.",  
    "Birds nest on both {m1} and {m2} during the summer.",  
    "Tour guides often describe the differences between {m1} and {m2}."
]

In [42]:
mountain_pairs = []

# Getting 7000 random mountain pairs
for i in range(7000):
    pair = random.sample(mountains, 2)
    if pair not in mountain_pairs:
        mountain_pairs.append(pair)


# For each pair 3 random templates are choosed. So we have 21000 sentences more
for pair in mountain_pairs:
    for t in random.sample(two_mountains_template, 3):
        text = t.format(m1=pair[0], m2 = pair[1])
        start1 = text.index(pair[0])
        end1 = start1 + len(pair[0])
        start2 = text.index(pair[1])
        end2 = start2 + len(pair[1])
        dataset.append({
            "text": text,
            "entities": [{"start": start1, "end": end1, "label": "MOUNTAIN"}, {"start": start2, "end": end2, "label": "MOUNTAIN"}]
        })

In [43]:
# 25 templates (most generated by ChatGPT) for creating sentences with three mountain names
three_mountains_template = [
    "I climbed the {m1} and {m2} last year, so {m3} is my next goal.",
    "What is the highest mountain among {m1}, {m2} and {m3}",
    "I recommend you to visit {m1} and {m2} this year, but {m3} will be not a good choice.",
    "Every rock climber should now about {m1}, {m2} and {m3}.",
    "{m1} and {m2} are not for beginners to climb, you should start with {m3}.",
    "Many travelers plan routes that include {m1}, {m2}, and {m3}.",
    "The view from {m1}, {m2}, and {m3} is unforgettable in different ways.",
    "Among {m1}, {m2}, and {m3}, only one has been climbed in winter.",
    "Legends connect {m1}, {m2}, and {m3} as the homes of ancient spirits.",
    "Snow from {m1}, {m2}, and {m3} feeds rivers that flow across the entire region.",
    "Climbers often try to summit {m1}, {m2}, and {m3} in one expedition.",
    "The peaks of {m1}, {m2}, and {m3} can be seen from the valley below.", 
    "Weather conditions on {m1}, {m2}, and {m3} are very different.",  
    "Hiking trails connecting {m1}, {m2}, and {m3} are challenging.",  
    "The wildlife around {m1}, {m2}, and {m3} is diverse and protected.",
    "After climbing {m1} last summer, I decided to attempt {m2}, and {m3} will be my next challenge.",
    "{m1} is known for its steep cliffs, while {m2} has easier paths, and {m3} is somewhere in between.",
    "Many hikers prefer {m1} first, then {m2}, and they always save {m3} for the final ascent.",
    "I took photos on {m1} and {m2}, but the view from {m3} impressed me the most.",
    "Local guides often recommend starting at {m1}, passing through {m2}, and ending at {m3}.",
    "While {m1} is covered in snow, {m2} is rocky, and {m3} has lush greenery.",
    "Climbing {m1} and {m2} requires careful planning; reaching {m3} demands even more experience.",
    "The rivers near {m1} and {m2} converge before reaching {m3}.",
    "Expeditions usually start at {m1}, cross {m2}, and finally summit {m3}.",
    "Weather on {m1} was perfect, {m2} was stormy, and {m3} surprised us with fog."
]

In [44]:
mountain_triples = []

# Getting 1200 random mountain triples
for i in range(1200):
    triple = random.sample(mountains, 3)
    if triple not in mountain_triples:
        mountain_triples.append(triple)


# For each triple 3 random templates are choosed. So we have 3600 sentences more
for triple in mountain_triples:
    for t in random.sample(three_mountains_template, 3):
        text = t.format(m1=triple[0], m2 = triple[1], m3 = triple[2])
        start1 = text.index(triple[0])
        end1 = start1 + len(triple[0])
        start2 = text.index(triple[1])
        end2 = start2 + len(triple[1])
        start3 = text.index(triple[2])
        end3 = start3 + len(triple[2])
        dataset.append({
            "text": text,
            "entities": [{"start": start1, "end": end1, "label": "MOUNTAIN"}, {"start": start2, "end": end2, "label": "MOUNTAIN"}, {"start": start3, "end": end3, "label": "MOUNTAIN"}]
        })

Now for creating balanced dataset we must create sentences without mountain names. So we are getting 3000 cities, 3000 rivers, 3000 islands, 2500 artists and 2500 mathematicians from Wikipedia to work with

In [45]:
query_cities = """
SELECT ?cityLabel WHERE {
  ?city wdt:P31/wdt:P279* wd:Q515.   # instance of city or subclass
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 3000
"""
cities = get_entities(query_cities)

In [46]:
query_rivers = """
SELECT ?riverLabel WHERE {
  ?river wdt:P31/wdt:P279* wd:Q4022.  # instance of river
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 3000
"""
rivers = get_entities(query_rivers)

In [47]:
query_islands = """
SELECT ?islandLabel WHERE {
  ?island wdt:P31/wdt:P279* wd:Q23442.  # instance of island
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 3000
"""
islands = get_entities(query_islands)

In [48]:
query_artists = """
SELECT ?artistLabel WHERE {
  ?artist wdt:P106 wd:Q483501.  # occupation: artist
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 2500
"""
artists = get_entities(query_artists)

In [49]:
query_mathematicians = """
SELECT ?mathematicianLabel WHERE {
  ?mathematician wdt:P31 wd:Q5;         # instance of human
                wdt:P106/wdt:P279* wd:Q170790.  # occupation: mathematician or subclass
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 2500
"""
mathematicians = get_entities(query_mathematicians)

Creating sentences without mountain names. 4 sentences for each city, river, island, artist and mathematician. We also check if something has the same name as mountain

In [50]:
# 30 templates (most generated by ChatGPT) for creating sentences with one city (without mountains)
cities_template = [
    "{c} is the beautiful place to live in.",
    "You must visit {c} at least once in life!",
    "{c} has rich history.",
    "You can't find another city such as {c}.",
    "Many conquerors tried to take {c}, but no one succeded.",
    "There are so many good places in {c}.",
    "I want to move in to {c}.",
    "{c} is pretty old city.",
    "There is a mountain not far from {c}.",
    "People from all over the world come to {c} every year.",
    "The architecture of {c} combines modern and ancient styles.",
    "Food in {c} is known for its unique and delicious taste.",
    "Life in {c} is always full of energy and movement.",
    "Artists often find inspiration while walking through {c}.",
    "The streets of {c} are especially beautiful at night.",
    "Public transport in {c} makes traveling around the city easy.",
    "Many famous festivals take place in {c} every year.",
    "The river flowing through {c} adds special charm to the city.",
    "{c} has a vibrant nightlife that attracts many visitors.",
    "Many tourists enjoy walking through the historic streets of {c}.",
    "The parks in {c} are perfect for relaxing on weekends.",
    "Shopping in {c} offers a mix of traditional and modern experiences.",
    "The museums of {c} display priceless works of art and history.",
    "Local markets in {c} are filled with colorful goods and crafts.",
    "The skyline of {c} is a combination of old towers and modern skyscrapers.",
    "Festivals in {c} celebrate both tradition and contemporary culture.",
    "The cafes of {c} are famous for their cozy atmosphere.",
    "Cycling through {c} is a popular way to explore the city.",
    "Historical monuments in {c} tell stories of its rich past.",
    "Many film and photo shoots take place in the scenic areas of {c}."
]

In [51]:
# For each city 4 random templates are choosed. So we have about 12000 sentences more
for c in cities:
    if c not in mountains:
        for t in random.sample(cities_template, 4):
            text = t.format(c=c)
            dataset.append({
                "text": text,
                "entities": []
            })

In [52]:
# 30 templates (most generated by ChatGPT) for creating sentences with one river (without mountains)
rivers_template = [
    "I love to swim in {r}.",
    "It is so quite and peacefully near {r}.",
    "{r} is so clean and nice.",
    "I go to {r} at least two times per month.",
    "We will go to {r} tomorrow.",
    "My father went fishing to {r} this morning.",
    "{r} is the most popular place in city for children to play.",
    "Wanna go to {r} next week?",
    "We must watch for {r} or it will become polluted.",
    "Many villages were built along the banks of {r}.",
    "The bridge across {r} gives a wonderful view of the sunset.",
    "Boats often travel along {r} during the summer.",
    "People like to have picnics near {r} on weekends.",
    "The sound of {r} helps me relax after a long day.",
    "Legends say that {r} never freezes, even in the coldest winter.",
    "Farmers use water from {r} to grow their crops.",
    "Several small streams flow into {r} before it reaches the sea.",
    "Wild animals often come to {r} to drink water.",
    "Fishermen catch fish from {r} early in the morning.",
    "The banks of {r} are lined with beautiful trees.",
    "Kayakers enjoy navigating the rapids of {r}.",
    "Birds often nest along the shores of {r}.",
    "During summer, families swim in the calm parts of {r}.",
    "The water of {r} reflects the surrounding mountains perfectly.",
    "Local legends speak about a hidden treasure near {r}.",
    "The sound of flowing water from {r} fills the valley.",
    "People often walk along the paths next to {r}.",
    "Bridges over {r} connect several nearby towns.",
    "Boats carrying goods often sail down {r}.",
    "Photographers love capturing sunsets by {r}."
]

In [53]:
# For each river 4 random templates are choosed. So we have about 12000 sentences more
for r in rivers:
    if r not in mountains:
        for t in random.sample(rivers_template, 4):
            text = t.format(r=r)
            dataset.append({
                "text": text,
                "entities": []
            })

In [54]:
# 30 templates (most generated by ChatGPT) for creating sentences with one island (without mountains)
islands_template = [
    "{i} is pretty big island.",
    "I have been living in {i} since I was six years old.",
    "I highly recommend you to visit {i}.",
    "{i} has interesting history.",
    "We are going to {i} on this weekend!",
    "My parents are from {i} too.",
    "{i} has very beautiful and generous people.",
    "I always wanted to visit {i}.",
    "Is there a high mountain on {i}?",
    "The beaches of {i} are known for their white sand and clear water.",
    "Many rare animals can be found only on {i}.",
    "People on {i} celebrate unique traditional festivals every year.",
    "Fishing is one of the main occupations on {i}.",
    "The air on {i} feels fresh and full of sea breeze.",
    "Tourists often explore {i} by renting small boats.",
    "The sunsets on {i} are some of the most beautiful in the world.",
    "{i} is surrounded by coral reefs full of colorful fish.",
    "After visiting {i}, I didn't want to leave.",
    "{i} is famous for its crystal-clear waters and sandy shores.", 
    "Many travelers enjoy hiking through the forests of {i}.",
    "Local cuisine on {i} is a blend of traditional and modern flavors.",
    "The cliffs of {i} offer stunning panoramic views of the ocean.",
    "Boats often take tourists around {i} to explore hidden coves.",
    "Birdwatchers visit {i} to see rare and exotic species.",
    "The villages on {i} are full of colorful houses and friendly locals.",
    "Snorkeling around {i} reveals vibrant coral reefs and fish.",
    "During festivals, {i} comes alive with music, dance, and food.",
    "The sunrise over {i} is a breathtaking sight every morning.",
    "People often camp on {i} to enjoy the peaceful night sky.",
    "The trails on {i} lead to waterfalls, caves, and hidden beaches."
]

In [55]:
# For each island 4 random templates are choosed. So we have about 12000 sentences more
for i in islands:
    if i not in mountains:
        for t in random.sample(islands_template, 4):
            text = t.format(i=i)
            dataset.append({
                "text": text,
                "entities": []
            })

In [56]:
# 30 templates (most generated by ChatGPT) for creating sentences with one artist (without mountains)
artists_template = [
    "{a} has so many beautiful paintings.",
    "{a} lived his all life in one city.",
    "Did you know {a} was also a writer?",
    "{a} has many hobbies, one of which was rock climbing.",
    "He learned it from {a}.",
    "I think {a} deserves more recognition.",
    "What do you think about {a}?",
    "I want to buy painting painted by {a}",
    "{a} was afraid of height.",
    "{a}'s works are displayed in museums all over the world.",
    "Many students were inspired by {a}'s techniques.",
    "{a} experimented with different styles throughout his career.",
    "Critics often praise {a} for originality and creativity.",
    "{a} held several exhibitions in Europe and Asia.",
    "Few people know that {a} also composed music.",
    "The style of {a} influenced many modern artists.",
    "Collectors pay high prices for paintings by {a}.",
    "{a} often signed his works with a unique symbol.",
    "{a} started painting at a very young age.",
    "Many museums dedicate entire galleries to {a}'s masterpieces.",  
    "{a} traveled extensively to study different art techniques.",  
    "Art students often study the works of {a} to learn new methods.",  
    "The themes of {a}'s paintings often reflect nature and human emotion.",  
    "Collectors from around the world seek out works by {a}.",  
    "Some of {a}'s paintings have become iconic symbols in art history.",  
    "Exhibitions featuring {a} attract thousands of visitors each year.",  
    "{a} often used vibrant colors to express deep feelings.",  
    "Many biographies have been written about the life of {a}.", 
    "Art critics frequently analyze {a}'s techniques and style.",  
    "Young artists often cite {a} as a major inspiration for their work."
]

In [57]:
# For each artist 4 random templates are choosed. So we have about 10000 sentences more
for a in artists:
    if a not in mountains:
        for t in random.sample(artists_template, 4):
            text = t.format(a=a)
            dataset.append({
                "text": text,
                "entities": []
            })

In [58]:
# 30 templates (most generated by ChatGPT) for creating sentences with one mathematician (without mountains)
mathematicians_template = [
    "You must know about {m}.",
    "{m} was also a good physicist.",
    "Do you know about {m}?",
    "{m} lived near high mountain.",
    "Works of {m} are good for learning maths.",
    "{m} was also good at chess.",
    "No one loved maths as much as {m} did.",
    "In my opinion, {m} is underrated",
    "{m} was a genius for sure.",
    "{m}'s theories are still studied in universities today.",
    "Many students were inspired by {m}'s work in mathematics.",
    "{m} contributed to several important discoveries.",
    "Historians often praise {m} for his dedication and intellect.",
    "{m} wrote books that are still referenced by scholars.",
    "Few people know that {m} also enjoyed philosophy.",
    "The legacy of {m} influenced many modern mathematicians.",
    "Collectors value original manuscripts by {m}.",
    "{m} solved problems that had puzzled mathematicians for centuries.",
    "{m} made groundbreaking contributions to number theory.",
    "Many research papers cite the work of {m}.",
    "{m} developed theories that are still relevant today.",
    "Students often study the methods introduced by {m}.",
    "The life of {m} is a source of inspiration for aspiring mathematicians.",
    "{m} collaborated with other famous scientists of his time.",
    "Conferences are sometimes held to discuss the work of {m}.",
    "Mathematical proofs by {m} are still considered masterpieces.",
    "{m} had a unique approach to solving complex problems.",
    "Biographies about {m} highlight both his work and personal life.",
    "The contributions of {m} span multiple fields of mathematics.", 
    "{m} received numerous awards and recognitions for his achievements."
]

In [59]:
# For each mathematician 4 random templates are choosed. So we have about 10000 sentences more
for m in mathematicians:
    if m not in mountains:
        for t in random.sample(mathematicians_template, 4):
            text = t.format(m=m)
            dataset.append({
                "text": text,
                "entities": []
            })

In [60]:
print(f"Dataset contains {len(dataset)} rows")

Dataset contains 160480 rows


Split data into training, test and validation samples

In [61]:
dataset = Dataset.from_list(dataset)

dataset = dataset.train_test_split(test_size=0.2)
test_valid = dataset["test"].train_test_split(test_size=0.5)

dataset = DatasetDict({
    "train": dataset["train"],
    "validation": test_valid["train"],
    "test": test_valid["test"]
})

Save dataset as JSONL so we don't need to generate it everytime

In [62]:
dataset["train"].to_json("train.jsonl")
dataset["validation"].to_json("validation.jsonl")
dataset["test"].to_json("test.jsonl")

Creating json from Arrow format:   0%|          | 0/129 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

1916936